1. Что бы не было проблем с загрузкой файл из-за ограничений колаба (+не загружать на гугл диск), настраиваю автоматическое скачивание датасета. Желательно по очереди перезапустить каждый блок.

In [1]:
! pip install kaggle
! sudo rm ~/.kaggle/kaggle.json
! sudo rmdir ~/.kaggle
! sudo mkdir ~/.kaggle

rm: cannot remove '/root/.kaggle/kaggle.json': No such file or directory
rmdir: failed to remove '/root/.kaggle': No such file or directory


In [2]:
# То что ключ лежит таким образом, понимаю что не безопасно.

! sudo echo '{"username":"rup1990","key":"c2c5c0c77d20a755667fd4153bda647b"}' > ~/.kaggle/kaggle.json

In [3]:
! sudo chmod 600 ~/.kaggle/kaggle.json

In [4]:
! kaggle datasets download yasserh/titanic-dataset
! unzip -d /usr/ -o titanic-dataset.zip

  0% 0.00/22.0k [00:00<?, ?B/s]
100% 22.0k/22.0k [00:00<00:00, 31.3MB/s]
Archive:  titanic-dataset.zip
  inflating: /usr/Titanic-Dataset.csv  


2. Тут начинается обработка файла

In [5]:
import numpy as np
import pandas as pd
import csv
import json
import matplotlib.pyplot as plt

In [6]:
# Загружаем файл в датафрейм

filename = "/usr/Titanic-Dataset.csv"
titanik_ds = pd.DataFrame(pd.read_csv (filename))
titanik_ds.set_index('PassengerId', inplace=True)

# Выведем набор данных

titanik_ds


,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S


In [7]:
# Считаем количество пустых ячеек

empty_cells_count = titanik_ds.isnull().sum()
empty_cells_count

Survived      0
Pclass        0
Name          0
Sex           0
Age         177
SibSp         0
Parch         0
Ticket        0
Fare          0
Cabin       687
Embarked      2
dtype: int64

In [8]:
# Вариант1. Заполним пустые значения

titanik_ds_filled = titanik_ds.copy()
titanik_ds_filled['Age'].fillna(method='ffill', inplace=True)
titanik_ds_filled['Cabin'].fillna('In the pantry', inplace=True)
titanik_ds_filled['Embarked'].fillna(method='ffill', inplace=True)
titanik_ds_filled

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,In the pantry,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,In the pantry,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,In the pantry,S
...,...,...,...,...,...,...,...,...,...,...,...
887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,In the pantry,S
888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,19.0,1,2,W./C. 6607,23.4500,In the pantry,S


In [9]:
# Вариант2. Удаляем пустые строки, в которых отсутствует хотя бы одно значение по колонкам age и Embarked
"""Объясняется тем, что правильнее удалять строки а не заполнять их синтетичекими данными, так как это влияет на качество данных.
У нас есть три колонки с пропусками, но очищаю только по двум, так как номер кают без данных о её размещении не явлюются ценными данными.
Предполагаю что стоит исключить из анализа эту колонку """

titanik_ds_dropped = titanik_ds.copy()
titanik_ds_dropped = titanik_ds_dropped.dropna(subset=['Age','Embarked'])
titanik_ds_dropped

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
886,0,3,"Rice, Mrs. William (Margaret Norton)",female,39.0,0,5,382652,29.1250,NaN,Q
887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S


In [10]:
del titanik_ds_dropped['Cabin']
titanik_ds_dropped

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
PassengerId,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S
...,...,...,...,...,...,...,...,...,...,...
886,0,3,"Rice, Mrs. William (Margaret Norton)",female,39.0,0,5,382652,29.1250,Q
887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,S
888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,S


In [11]:
# Проверяем что не осталось пустых ячеек (Вариант 1)

empty_cells_count_v1 = titanik_ds_filled.isnull().sum()
empty_cells_count_v1

Survived    0
Pclass      0
Name        0
Sex         0
Age         0
SibSp       0
Parch       0
Ticket      0
Fare        0
Cabin       0
Embarked    0
dtype: int64

In [12]:
# Проверяем что не осталось пустых ячеек (Варианту 2)

empty_cells_count_v2 = titanik_ds_dropped.isnull().sum()
empty_cells_count_v2

Survived    0
Pclass      0
Name        0
Sex         0
Age         0
SibSp       0
Parch       0
Ticket      0
Fare        0
Embarked    0
dtype: int64

In [13]:
# Проверяем количество уникальных значений по каждаму параметру (Вариант 1)

unics_values = titanik_ds_filled.nunique()
unics_values

Survived      2
Pclass        3
Name        891
Sex           2
Age          88
SibSp         7
Parch         7
Ticket      681
Fare        248
Cabin       148
Embarked      3
dtype: int64

In [14]:
# Проверяем количество уникальных значений по каждаму параметру по (Вариант 2)

unics_values = titanik_ds_dropped.nunique()
unics_values

Survived      2
Pclass        3
Name        712
Sex           2
Age          88
SibSp         6
Parch         7
Ticket      541
Fare        219
Embarked      3
dtype: int64

In [15]:
# Проверяем минимальное значения для каждого параметра (Вариант 1)

min_val = titanik_ds_filled.min()
min_val

Survived                      0
Pclass                        1
Name        Abbing, Mr. Anthony
Sex                      female
Age                        0.42
SibSp                         0
Parch                         0
Ticket                   110152
Fare                        0.0
Cabin                       A10
Embarked                      C
dtype: object

In [16]:
# Проверяем минимальное значения для каждого параметра (Вариант 2)

min_val = titanik_ds_dropped.min(numeric_only=None)
min_val

Survived                      0
Pclass                        1
Name        Abbing, Mr. Anthony
Sex                      female
Age                        0.42
SibSp                         0
Parch                         0
Ticket                   110152
Fare                        0.0
Embarked                      C
dtype: object

In [17]:
# Проверяем максимальное значения для каждого параметра (Вариан 1)

max_val = titanik_ds_filled.max()
max_val

Survived                              1
Pclass                                3
Name        van Melkebeke, Mr. Philemon
Sex                                male
Age                                80.0
SibSp                                 8
Parch                                 6
Ticket                        WE/P 5735
Fare                           512.3292
Cabin                                 T
Embarked                              S
dtype: object

In [18]:
# Проверяем максимальное значения для каждого параметра (Вариант 2)

max_val = titanik_ds_dropped.max()
max_val

Survived                                  1
Pclass                                    3
Name        van Billiard, Mr. Austin Blyler
Sex                                    male
Age                                    80.0
SibSp                                     5
Parch                                     6
Ticket                            WE/P 5735
Fare                               512.3292
Embarked                                  S
dtype: object

In [19]:
# Рассчитаем коэффициенты корреляции между параметром выжываемости и остальными числовыми параметрами (Вариант 1)

correlation_coef = titanik_ds_filled.corr(numeric_only = True)['Survived']
correlation_coef

Survived    1.000000
Pclass     -0.338481
Age        -0.065571
SibSp      -0.035322
Parch       0.081629
Fare        0.257307
Name: Survived, dtype: float64

In [20]:
# Рассчитаем коэффициенты корреляции между параметром выжываемости и остальными числовыми параметрами (Вариант 2)

correlation_coef = titanik_ds_dropped.corr(numeric_only = True)['Survived']
correlation_coef

Survived    1.000000
Pclass     -0.356462
Age        -0.082446
SibSp      -0.015523
Parch       0.095265
Fare        0.266100
Name: Survived, dtype: float64

In [21]:
# Группируем значения, что бы понять среднее влияние каждого значения (Вариант 1)
"""Исключил некоторые параметры, которые, как мне показалось, не влияют на конечный результат"""

grouped_data_sex = titanik_ds_filled.groupby('Sex')['Survived'].mean()
grouped_data_pclass = titanik_ds_filled.groupby('Pclass')['Survived'].mean()
grouped_data_age = titanik_ds_filled.groupby('Age')['Survived'].mean()
grouped_data_sibsp = titanik_ds_filled.groupby('SibSp')['Survived'].mean()
grouped_data_parch = titanik_ds_filled.groupby('Parch')['Survived'].mean()
grouped_data_fare = titanik_ds_filled.groupby('Fare')['Survived'].mean()
grouped_data_embarked  = titanik_ds_filled.groupby('Embarked')['Survived'].mean()
grouped_data_sex, grouped_data_pclass, grouped_data_age, grouped_data_sibsp, grouped_data_parch, grouped_data_fare, grouped_data_embarked

(Sex
 female    0.742038
 male      0.188908
 Name: Survived, dtype: float64,
 Pclass
 1    0.629630
 2    0.472826
 3    0.242363
 Name: Survived, dtype: float64,
 Age
 0.42     1.000000
 0.67     1.000000
 0.75     0.666667
 0.83     0.666667
 0.92     1.000000
            ...   
 70.00    0.000000
 70.50    0.000000
 71.00    0.000000
 74.00    0.000000
 80.00    1.000000
 Name: Survived, Length: 88, dtype: float64,
 SibSp
 0    0.345395
 1    0.535885
 2    0.464286
 3    0.250000
 4    0.166667
 5    0.000000
 8    0.000000
 Name: Survived, dtype: float64,
 Parch
 0    0.343658
 1    0.550847
 2    0.500000
 3    0.600000
 4    0.000000
 5    0.200000
 6    0.000000
 Name: Survived, dtype: float64,
 Fare
 0.0000      0.066667
 4.0125      0.000000
 5.0000      0.000000
 6.2375      0.000000
 6.4375      0.000000
               ...   
 227.5250    0.750000
 247.5208    0.500000
 262.3750    1.000000
 263.0000    0.500000
 512.3292    1.000000
 Name: Survived, Length: 248, dtype: fl

In [22]:
# Группируем значения, что бы понять среднее влияние каждого значения (Вариант 2)
"""Исключил некоторые параметры, которые, как мне показалось, не влияют на конечный результат"""

grouped_data_sex = titanik_ds_dropped.groupby('Sex')['Survived'].mean()
grouped_data_pclass = titanik_ds_dropped.groupby('Pclass')['Survived'].mean()
grouped_data_age = titanik_ds_dropped.groupby('Age')['Survived'].mean()
grouped_data_sibsp = titanik_ds_dropped.groupby('SibSp')['Survived'].mean()
grouped_data_parch = titanik_ds_dropped.groupby('Parch')['Survived'].mean()
grouped_data_fare = titanik_ds_dropped.groupby('Fare')['Survived'].mean()
grouped_data_embarked  = titanik_ds_dropped.groupby('Embarked')['Survived'].mean()
grouped_data_sex, grouped_data_pclass, grouped_data_age, grouped_data_sibsp, grouped_data_parch, grouped_data_fare, grouped_data_embarked

(Sex
 female    0.752896
 male      0.205298
 Name: Survived, dtype: float64,
 Pclass
 1    0.652174
 2    0.479769
 3    0.239437
 Name: Survived, dtype: float64,
 Age
 0.42     1.0
 0.67     1.0
 0.75     1.0
 0.83     1.0
 0.92     1.0
         ... 
 70.00    0.0
 70.50    0.0
 71.00    0.0
 74.00    0.0
 80.00    1.0
 Name: Survived, Length: 88, dtype: float64,
 SibSp
 0    0.368870
 1    0.530055
 2    0.440000
 3    0.333333
 4    0.166667
 5    0.000000
 Name: Survived, dtype: float64,
 Parch
 0    0.354528
 1    0.554545
 2    0.573529
 3    0.600000
 4    0.000000
 5    0.200000
 6    0.000000
 Name: Survived, dtype: float64,
 Fare
 0.0000      0.142857
 4.0125      0.000000
 5.0000      0.000000
 6.2375      0.000000
 6.4375      0.000000
               ...   
 227.5250    1.000000
 247.5208    0.500000
 262.3750    1.000000
 263.0000    0.500000
 512.3292    1.000000
 Name: Survived, Length: 219, dtype: float64,
 Embarked
 C    0.607692
 Q    0.285714
 S    0.362816
 Name: S